In [154]:
import numpy as np
import queue
arr = np.genfromtxt('Day17.txt',delimiter = ',',dtype=np.int64)

In [1]:
def indexing(listt,index):
    inxlist = []
    global arr1
    for i, ss in enumerate(listt):
        if ss == '0': #position mode
            inxlist.append(arr1[index+i+1])
        elif ss =='1': #immediate mode
            inxlist.append(index+i+1)
        else: #ss == '2'
            global base
            inxlist.append(arr1[index+i+1]+base)
    return inxlist

def arrOdict(inxlist):
    global arr1
    global lgmem
    vallist = []
    for inx in inxlist:
        if inx > (len(arr1)-1):
            if inx not in lgmem.keys():
                lgmem[inx] = 0
                vallist.append(lgmem[inx])
            else:
                vallist.append(lgmem[inx])
        else:
            vallist.append(arr1[inx])
    return vallist

def setequal(inxequal,num):
    global arr1
    global lgmem
    if inxequal > (len(arr1)-1):
        lgmem[inxequal] = num
    else:
        arr1[inxequal] = num
    return None

def opcodeAct(index,inxlist,opcode):
    global arr1
    global inpt
    global asciiQ
    vallist = arrOdict(inxlist)
    if opcode == 1:
        setequal(inxlist[2],vallist[0] + vallist[1])
        index += 4
    elif opcode == 2:
        setequal(inxlist[2],vallist[0] * vallist[1])
        index += 4
    elif opcode == 4:
        asciiQ.put(vallist[0])
        index += 2
    elif opcode == 3:
        setequal(inxlist[0],inpt)
        index += 2
    elif opcode == 5:
        if vallist[0]: #If non-zero is true
            index = vallist[1]
        else:
            index += 3
    elif opcode == 6:
        if not vallist[0]:
            index = vallist[1]
        else:
            index += 3
    elif opcode == 7:
        if vallist[0] < vallist[1]:
            setequal(inxlist[2],1)
        else:
            setequal(inxlist[2],0)
        index += 4
    elif opcode == 8:
        if vallist[0] == vallist[1]:
            setequal(inxlist[2],1)
        else:
            setequal(inxlist[2],0)
        index += 4
    elif opcode == 9:
        global base
        base += vallist[0]
        index += 2
    return index

def day9(arr):
    global arr1
    global inpt
    global dictMap
    global asciiQ
    arr1 = arr.copy()
    index = 0
    asciiQ = queue.Queue()
    asciiList = []
    dictMap_counter = 0
    while arr1[index] != 99:
        if arr1[index] in [1,2,7,8]:
            index = opcodeAct(index,arr1[index+1:index+4],arr1[index])
        elif arr1[index] in [5,6]:
            index = opcodeAct(index,arr1[index+1:index+3],arr1[index])
        elif arr1[index] in [3,4,9]:
            index = opcodeAct(index,arr1[index+1:index+2],arr1[index])
        else: #5 digit opcode
            s = str(arr1[index])
            s = s.rjust(5,'0')
            s = s[::-1]
            if int(s[0]) in [1,2,7,8]:
                inxlist = indexing(s[2:],index)
            elif int(s[0]) in [5,6]:
                inxlist = indexing(s[2:4],index)
            elif int(s[0]) in [3,4,9]:
                inxlist = indexing(s[2:3],index)
            index = opcodeAct(index,inxlist,int(s[0]))
        if len(asciiQ.queue) == 1:
            asciiV = asciiQ.get()
            if asciiV == 35: #1 = Scaffold
                asciiList.append(1)
            elif asciiV == 46: #0 = Open Space
                asciiList.append(0)
            elif asciiV == 94:
                asciiList.append(1)
            elif asciiV == 10:
                dictMap[dictMap_counter] = asciiList
                asciiList = []
                dictMap_counter += 1
    return None

In [285]:
base = 0
lgmem = {}
inpt = 2
dictMap = {}
day9(arr)

In [279]:
dictMap_list = [value for value in dictMap.values() if value != []]
dictMap_arr = np.array(dictMap_list)
dictMap_arr_pad = np.pad(dictMap_arr,[1,1],mode='constant')

In [280]:
row_len = range(1,dictMap_arr_pad.shape[0]-1)
column_len = range(1,dictMap_arr_pad.shape[1]-1)

In [281]:
dictMap_inter = np.zeros(dictMap_arr.shape)
for row in row_len:
    for col in column_len:
        dictMap_inter[row-1,col-1] = (dictMap_arr_pad[row-1,col]+dictMap_arr_pad[row+1,col]+
                        dictMap_arr_pad[row,col-1]+dictMap_arr_pad[row,col+1]+
                         dictMap_arr_pad[row,col])

In [282]:
inter_index = np.argwhere(dictMap_inter == 5)
np.sum((inter_index[:,0])*(inter_index[:,1]))

2660